In [1]:
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
import nbfigtulz as ftl

import numpy as np
import pandas as pd
import scipy.stats

from tqdm import tqdm

This notebook renders our visualizations of the Monocular Depth Estimation experiment. We used the cache file `cached_depth_results.pkl` generated by [`gen_depth_results.py`](https://github.com/aamini/evidential-deep-learning/blob/d1d8e395fb083308d14fa92c5ce766e97b2a066a/neurips2020/gen_depth_results.py) after changing [line 436](https://github.com/aamini/evidential-deep-learning/blob/d1d8e395fb083308d14fa92c5ce766e97b2a066a/neurips2020/gen_depth_results.py#L436) accordingly to reflect our redefintions of $u'_\text{al}$ and $u'_\text{ep}$.

Similarly, we use the Dropout model with a dropout probability set to zero to fit a vanilla Gaussian NLL to the data. Conveniently, most of this is already implemented in [github.com/aamini/evidential-deep-learning/](https://github.com/aamini/evidential-deep-learning/blob/d1d8e395fb083308d14fa92c5ce766e97b2a066a/neurips2020/trainers/gaussian.py):
 - [models/depth/gaussian.py](https://github.com/aamini/evidential-deep-learning/blob/d1d8e395fb083308d14fa92c5ce766e97b2a066a/neurips2020/models/depth/gaussian.py)
 - [trainers/gaussian.py](https://github.com/aamini/evidential-deep-learning/blob/d1d8e395fb083308d14fa92c5ce766e97b2a066a/neurips2020/trainers/gaussian.py)

In [2]:
EVIDENTIAL_FILES = [
    'cached_depth_results_original.pkl',  # orignal
    'cached_depth_results_al.pkl',        # new aleatoric
    'cached_depth_results_ep.pkl',        # new epistemic
]

GAUSSIAN_FILES = [
    'cached_depth_results_gaussian_26000.pkl',  # gaussian regression after n1 epochs
    'cached_depth_results_gaussian_45000.pkl',  # gaussian regression after n2 epochs
]
EPOCHS = ['26k', '45k']  # string representation of epochs n1 and n2

In [3]:
FIG_SIZE_SMALL = (3.0, 2.3)  # small images for paper
FIG_SIZE_LARGE = (4.0, 3.0)  # larger images for presentation
FIG_SIZE = FIG_SIZE_LARGE

In [4]:
img_dir = f'img/depth'
!rm -rf {img_dir}/ && mkdir {img_dir}/
ftl.config['img_dir'] = img_dir

print(f'Images are stored in: {img_dir}')

Images are stored in: img/depth


In [5]:
def flatten_df(df, keys=None):
    if keys is None:
        keys = []
    
    required_keys = ['Method',]
    keys = required_keys + keys
    max_shape = max([np.prod(np.shape(df[key].iloc[0])) for key in keys])

    contents = {}
    for key in keys:
        if np.prod(np.shape(df[key].iloc[0])) == 1:
            contents[key] = np.repeat(df[key], max_shape)
        else:
            contents[key] = np.stack(df[key], axis=0).flatten()

    return pd.DataFrame(contents)

In [6]:
def make_cutoff(pxl):
    df = pd.DataFrame(columns=['Method', 'Percentile', 'Error'])
    
    for method in pxl['Method'].unique():
        sel = (pxl['Method'] == method)
        df_sorted = pxl[sel].sort_values('Sigma', ascending=False)
            
        percentiles = np.arange(100) / 100.0
        idx = (percentiles * df_sorted.shape[0]).astype(int)

        error = np.abs(df_sorted['Mu'] - df_sorted['Target'])
        mean_error = [error[i:].mean() for i in idx]
        df = pd.concat((df, pd.DataFrame.from_dict({
            'Method': method,
            'Percentile': percentiles,
            'Error': mean_error,
        })), ignore_index=True)

    return df

In [7]:
def make_calibration(pxl):
    df = pd.DataFrame(columns=['Method', 'Expected Conf.', 'Observed Conf.'])

    expected_p = np.arange(41) / 40.0
    for method in pxl['Method'].unique():
        sel = (pxl['Method'] == method)
        x = pxl[sel]
        
        observed_p = [
            (x['Target'] < scipy.stats.norm.ppf(p, x['Mu'], x['Sigma'])).mean()
        for p in expected_p]

        df = pd.concat((df, pd.DataFrame.from_dict({
            'Method': method,
            'Expected Conf.': expected_p,
            'Observed Conf.': observed_p,
        })), ignore_index=True)

    return df

In [8]:
def make_ood_cmp(pxl):
    pxl = pxl[['Method', 'Sigma', 'OOD']].copy()
    pxl['Entropy'] = 0.5 * np.log(2.0 * np.pi * pxl["Sigma"]**2)

    mean = pxl.groupby([pxl.index, 'Method', 'OOD']).mean().reset_index()
    return mean[['Method', 'OOD', 'Entropy']].copy()

In [9]:
def parse_file(file_name, *, methods):
    df = pickle.load(open(file_name, 'rb'))
    df = df[(df['Epsilon'].abs() < 1e-8) & (df['Method'].isin(methods))]
        
    sel = (df['OOD'] == False)
    pxl = flatten_df(df[sel], keys=['Target', 'Mu', 'Sigma'])
    cutoff = make_cutoff(pxl)
    calib = make_calibration(pxl)
    
    pxl = flatten_df(df, keys=['Sigma', 'OOD'])
    oodcmp = make_ood_cmp(pxl)
    
    return cutoff, calib, oodcmp

In [10]:
cutoffs = dict()
calibs = dict()
oodcmps = dict()

In [11]:
for k, f in tqdm(zip(['original', 'al', 'ep'], EVIDENTIAL_FILES), total=3):
    cutoff, calib, oodcmp = parse_file(f, methods=('Evidential',)) 
    cutoffs[k] = cutoff
    calibs[k] = calib
    oodcmps[k] = oodcmp

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:05<00:00, 41.86s/it]


In [12]:
for i, f in enumerate(tqdm(GAUSSIAN_FILES)):
    cutoff, calib, oodcmp = parse_file(f, methods=('Gaussian',))
    cutoffs[f'Gaussian {i + 1}'] = cutoff
    calibs[f'Gaussian {i + 1}'] = calib
    oodcmps[f'Gaussian {i + 1}'] = oodcmp

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.63s/it]


In [13]:
for k in ['original', 'al', 'ep']:
    cutoffs[k]['Method'] = cutoffs[k]['Method'] + f' {k}'
    calibs[k]['Method'] = calibs[k]['Method'] + f' {k}'
    oodcmps[k]['Method'] = oodcmps[k]['Method'] + f' {k}'

for k in ['Gaussian 1', 'Gaussian 2']:
    cutoffs[k]['Method'] = k
    calibs[k]['Method'] = k
    oodcmps[k]['Method'] = k
    
cutoffs = pd.concat([cutoffs[k] for k in cutoffs], ignore_index=True)
calibs = pd.concat([calibs[k] for k in calibs], ignore_index=True)
oodcmps = pd.concat([oodcmps[k] for k in oodcmps], ignore_index=True)

In [14]:
@ftl.with_context
def make_cutoff_plot(df):
    fig, ax = plt.subplots()
    ax.set_xlabel('CL (%)')
    ax.set_ylabel(r'$|y_i - \gamma_i|$')
    
    style = {
        'Evidential original': {'color': 'black', 'linestyle': '--', 'dashes': (3, 5), 'label': r'$u_\mathrm{ep}$'},
        'Evidential al': {'color': 'C1', 'label': r"$u'_\mathrm{al}$"},
        'Evidential ep': {'color': 'C0', 'label': r"$u'_\mathrm{ep}$"},
        'Gaussian 1': {'color': 'C2', 'label': r'$\sigma_\mathrm{' + f'{EPOCHS[0]}' + '}$'},
        'Gaussian 2': {'color': 'C3', 'label': r'$\sigma_\mathrm{' + f'{EPOCHS[1]}' + '}$'},
    }
    
    for m in [
        'Evidential ep',
        'Evidential al',
        'Gaussian 1',
        'Gaussian 2',
        'Evidential original']:
        
        sel = (df['Method'] == m)
        x = df[sel]['Percentile'] * 100
        y = df[sel]['Error']
        ax.plot(x, y, **style[m])
        
    ax.legend()
    
    return ftl.save_fig(fig, 'cutoffs', resize=FIG_SIZE)
            
            
make_cutoff_plot(cutoffs)

cutoffs.png

In [15]:
@ftl.with_context
def make_calibration_plot(df):
    fig, ax = plt.subplots()
    ax.set_xlabel('Expected CL (%)')
    ax.set_ylabel('Observed CL (%)')
    
    style = {
        'Evidential original': {'color': 'black', 'linestyle': '--', 'dashes': (3, 5), 'label': r'$u_\mathrm{ep}$'},
        'Evidential al': {'color': 'C1', 'label': r"$u'_\mathrm{al}$"},
        'Evidential ep': {'color': 'C0', 'label': r"$u'_\mathrm{ep}$"},
        'Gaussian 1': {'color': 'C2', 'label': r'$\sigma_\mathrm{' + f'{EPOCHS[0]}' + '}$'},
        'Gaussian 2': {'color': 'C3', 'label': r'$\sigma_\mathrm{' + f'{EPOCHS[1]}' + '}$'},
    }
    
    for m in ['Evidential ep', 'Evidential al', 'Gaussian 1', 'Gaussian 2', 'Evidential original']:
        sel = (df['Method'] == m)
        x = df[sel]['Expected Conf.'] * 100
        y = df[sel]['Observed Conf.'] * 100
        ax.plot(x, y, **style[m])
        
    # ax.legend()
    
    return ftl.save_fig(fig, 'calibs', resize=FIG_SIZE)
            
            
make_calibration_plot(calibs)

calibs.png

In [16]:
@ftl.with_context
def make_ood_boxes(df):
    methods = ['Evidential original', 'Evidential al', 'Gaussian 1', 'Gaussian 2']
    
    sel_id = [(df['Method'] == m) & (df['OOD'] == False) for m in methods]
    sel_ood = [(df['Method'] == m) & (df['OOD'] == True) for m in methods]
       
    entropy = df['Entropy']
    
    fig, ax = plt.subplots()
    ax.violinplot([entropy[s] for s in sel_id], showextrema=False, showmeans=True)
    ax.violinplot([entropy[s] for s in sel_ood], showextrema=False, showmeans=True)
    
    ax.legend([
        mpl.lines.Line2D([0], [0], color='C0', lw=4, alpha=.5),
        mpl.lines.Line2D([0], [0], color='C1', lw=4, alpha=.5),
    ], ['ID', 'OOD'], loc='lower right')
    
    labels = {
        'Evidential original': r'$u_\mathrm{ep}$',
        'Evidential al': r"$u'_\mathrm{al}$",
        'Gaussian 1': r'$\sigma_\mathrm{' + f'{EPOCHS[0]}' + '}$',
        'Gaussian 2': r'$\sigma_\mathrm{' + f'{EPOCHS[1]}' + '}$',
    }
    ax.set_xticks(ticks=range(1, len(methods) + 1), labels=[labels[m] for m in methods])
    ax.set_ylabel('Entropy')
    
    return ftl.save_fig(fig, 'entropy', resize=FIG_SIZE)
    
    
make_ood_boxes(oodcmps)

entropy.png